# [Module 4.3.2] inference on custem tf serving image


---
이 노트북은 약 10정도 시간이 소요 됩니다.

In [39]:
import boto3
import sagemaker
import pandas as pd
import os

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [40]:
%store -r 

In [41]:
# training_job_name = 'bert2tweet-2020-07-08-07-58-27-895'
print(training_job_name)

tensorflow-training-2020-08-02-08-19-13-279


In [42]:
inference_image = ecr_infer_custom_image_tf_serving_20_cpu
print("inference_image: ", inference_image)

inference_image:  343441690612.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-tensorflow-serving:2.0.0-cpu


## Model Creation with Custome TFS Docker Image and Inference code

In [43]:
from sagemaker.tensorflow.serving import Model

model = Model(model_data='s3://{}/{}/output/model.tar.gz'.format(bucket, training_job_name),
              role=role,
              entry_point='inference.py',
              image = inference_image
             ) 

In [44]:
print(model.image)
print(model.serving_image_uri)
print(model.model_data)
print(model.name)
print(model.FRAMEWORK_NAME)


343441690612.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-tensorflow-serving:2.0.0-cpu
<bound method Model.serving_image_uri of <sagemaker.tensorflow.serving.Model object at 0x7f7bc14e6b00>>
s3://sagemaker-ap-northeast-2-343441690612/tensorflow-training-2020-08-02-08-19-13-279/output/model.tar.gz
None
tensorflow-serving


## 엔드포인트 생성

In [45]:
%%time

instance_type='ml.m4.xlarge'
deployed_model = model.deploy(initial_instance_count = 1,
                             instance_type = instance_type,
                             wait=True)


-------------!CPU times: user 44.3 s, sys: 7.51 s, total: 51.8 s
Wall time: 7min 25s


## Predictor Creation on the Endpoint

In [46]:
# tweet_bert_endpoint_name = 'train_text, train_label, test_text, test_label = tweet_data.split_train_test_data(texts, labels, 0.9)
tweet_bert_endpoint_name = deployed_model.endpoint
print(tweet_bert_endpoint_name)

sagemaker-tensorflow-serving-2020-08-02-09-30-29-170


In [47]:
import json
from sagemaker.tensorflow.serving import Predictor

predictor = Predictor(endpoint_name = tweet_bert_endpoint_name,
                      sagemaker_session = sess,
                      content_type = 'application/json',
                     )

## Inference 실행

In [48]:
from TweetUtil import TweetUtil

tweet_util = TweetUtil()
tweet_util.load_emoji_data('emoji_to_idx.pickle')
emoji = tweet_util.get_emo_class_label(3)
print(emoji)

emoji_to_idx is loaded
😂


In [49]:
test_file_path = 'data/test/tweet_file_test.csv'
test_df = pd.read_csv(test_file_path)
test_file_path = 'data/test/tweet_file_test.csv'
test_df = pd.read_csv(test_file_path)
sample_df = test_df.sample(10)
sample_df

,TWEET,LABEL
7535,last day today so cast your votes now aldubks...,1
9205,wow you're really on fire today boarding a fu...,8
854,when tasha came through with her gun too on t...,2
316,the females that live in my house are looking ...,2
6483,wouldn't spend it any other way,0
6850,thank you love ya,1
5401,that's amazing love the light grey,0
4204,nascar motorsport jimmiejohnson t shirt l lar...,0
5834,life is too short to be organized,3
2215,happy endings there are no happy endings endi...,9


In [50]:
def show_top_N_label(score_list, topN):

    import numpy as np

    top_n_idx = np.argsort(score_list)[-topN:]
    top_n_values = [score_list[i] for i in top_n_idx]
    
    top_n_idx_list = top_n_idx.tolist()
    top_n_idx_list.reverse()
    top_n_values = [score_list[i] for i in top_n_idx_list]    
    
    return top_n_idx_list


## Top 3 이모티콘 추천

In [56]:
import tensorflow as tf
import json

columns = ['TWEET', 'LABEL']
topN = 3
for tweet, label in zip(sample_df.TWEET.values, sample_df.LABEL.values):
    # print("label: {}, tweet: {}".format(label, tweet))
    
    reviews = [tweet]
    
    print("reviews: \n", reviews)



#     predicted_classes = predictor.predict(reviews)[0]
    predicted_classes = predictor.predict(reviews)    
    # predicted_classes = show_top_N_label(predicted_classes, topN)

    print('tweet: {} \nGround_truth- {}:{}\n '.format(
        tweet,
        label, 
        tweet_util.get_emo_class_label(label))
         )    
    
    print(predicted_classes)
#     print('Prediction: {},{},{},{},{},{} \n '.format(
#         predicted_classes[0], 
#         tweet_util.get_emo_class_label(predicted_classes[0]),
#         predicted_classes[1], 
#         tweet_util.get_emo_class_label(predicted_classes[1]),
#         predicted_classes[2], 
#         tweet_util.get_emo_class_label(predicted_classes[2])                                       
#         ))    
        



reviews: 
 [' last day today so cast your votes now aldubksliveson']
tweet:  last day today so cast your votes now aldubksliveson 
Ground_truth- 1:💕
 
[[0.0415807478, -0.0668697134]]
reviews: 
 [" wow you're really on fire today boarding a full flight all at once after first so glad i have priority checkedbag"]
tweet:  wow you're really on fire today boarding a full flight all at once after first so glad i have priority checkedbag 
Ground_truth- 8:🙄
 
[[0.058101695, -0.113432437]]
reviews: 
 [' when tasha came through with her gun too on tatiyana wheewwww yasssss girl you better hold your man down powerfin']
tweet:  when tasha came through with her gun too on tatiyana wheewwww yasssss girl you better hold your man down powerfin 
Ground_truth- 2:🔥
 
[[0.0731375366, -0.0678214431]]
reviews: 
 ['the females that live in my house are looking af for this party']
tweet: the females that live in my house are looking af for this party 
Ground_truth- 2:🔥
 
[[0.0477781668, -0.0804672763]]
review